# Korean

In [1]:
! pip install numpy
! pip install kiwipiepy
! pip install nltk
! pip install datasets
! pip install stanza

### Load Dataset

In [2]:
from datasets import load_dataset
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize  # tokenization

nltk.download('punkt')       # Tokenizer model
nltk.download('punkt_tab')

nltk.download('stopwords')   # List of common stopwords for text normalization
from nltk.corpus import stopwords

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/park/miniforge3/envs/nlp_clean/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /Users/park/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/park/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/park/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
dataset_ko = load_dataset("wikimedia/wikipedia", "20231101.ko")

In [4]:
df_ko = pd.DataFrame(dataset_ko["train"])
df_sample_ko = df_ko.sample(n=1000, random_state=42)
df_sample_ko.head()

,id,url,title,text
623706,3582873,https://ko.wikipedia.org/wiki/%EC%B9%B4%ED%83%...,카타쿠라 카네타로,카타쿠라 카네타로 ()는 현재의 카타쿠라 공업의 대대로 이어진 이름이다\n\n 카타...
23304,70654,https://ko.wikipedia.org/wiki/%ED%83%84%EB%91%90,탄두,"탄두(彈頭, )란 미사일, 로켓, 어뢰 등의 추진에 의해 전달되는 폭발성 물질 또는..."
79240,268688,https://ko.wikipedia.org/wiki/%EB%8B%88%EC%8A%...,니스 (프랑스),"니스(, 니사르드 , , , , )는 프랑스 남부의 항만 도시로 프랑스의 지중해 연..."
140214,529155,https://ko.wikipedia.org/wiki/%EA%B9%80%ED%99%...,김홍수 게이트,김홍수 게이트는 90년대 초 초등학교 동창인 법조인 친구를 통해 서울지방법원에서 근...
28599,89713,https://ko.wikipedia.org/wiki/%EC%9D%B8%EB%8F%...,인도아리아어군의 언어 목록,다음은 인도아리아어군의 언어 목록이다.\n\n\n 고전어\n 베다 산스크리트어\n ...


### Removing punctuations and symbols

In [5]:
def clean_text_ko(text):
    text = re.sub(r'\n', ' ', text)  # Remove newline characters
    text = re.sub(r"[^가-힣0-9\s]", "", text) # Keep only Korean chars, digits, and spaces
    text = re.sub(r"[ㄱ-ㅎㅏ-ㅣ]", "", text) # Remove isolated Characters
    text = re.sub(r'\s+', ' ', text).strip()  # multiple spaces
    return text

df_sample_ko["clean_text"] = df_sample_ko["text"].apply(clean_text_ko)

### Tokenization

In [6]:
from kiwipiepy import Kiwi

# Kiwi.analyze returns morphemes and POS tags
kiwi = Kiwi()

def tokenize(text):
    # top_n=1: use the most probable analysis (faster)
    analyzed = kiwi.analyze(text, top_n=1)
    # analyzed[0][0]: list of (morph, POS, start, length)
    tokens = [m[0] for m in analyzed[0][0]]
    return tokens

df_sample_ko["tokens"] = df_sample_ko["clean_text"].apply(tokenize)

In [7]:
df_sample_ko.head()

,id,url,title,text,clean_text,tokens
623706,3582873,https://ko.wikipedia.org/wiki/%EC%B9%B4%ED%83%...,카타쿠라 카네타로,카타쿠라 카네타로 ()는 현재의 카타쿠라 공업의 대대로 이어진 이름이다\n\n 카타...,카타쿠라 카네타로 는 현재의 카타쿠라 공업의 대대로 이어진 이름이다 카타쿠라 카네타...,"[카타쿠라 카네타로, 는, 현재, 의, 카타쿠라, 공업, 의, 대대, 로, 이어지,..."
23304,70654,https://ko.wikipedia.org/wiki/%ED%83%84%EB%91%90,탄두,"탄두(彈頭, )란 미사일, 로켓, 어뢰 등의 추진에 의해 전달되는 폭발성 물질 또는...",탄두 란 미사일 로켓 어뢰 등의 추진에 의해 전달되는 폭발성 물질 또는 독성 물질이...,"[탄두, 이, 란, 미사일, 로켓, 어뢰, 등, 의, 추진, 에, 의하, 어, 전달..."
79240,268688,https://ko.wikipedia.org/wiki/%EB%8B%88%EC%8A%...,니스 (프랑스),"니스(, 니사르드 , , , , )는 프랑스 남부의 항만 도시로 프랑스의 지중해 연...",니스 니사르드 는 프랑스 남부의 항만 도시로 프랑스의 지중해 연안에 위치해 있다 마...,"[니스, 니사르드, 는, 프랑스, 남부, 의, 항만, 도시, 로, 프랑스, 의, 지..."
140214,529155,https://ko.wikipedia.org/wiki/%EA%B9%80%ED%99%...,김홍수 게이트,김홍수 게이트는 90년대 초 초등학교 동창인 법조인 친구를 통해 서울지방법원에서 근...,김홍수 게이트는 90년대 초 초등학교 동창인 법조인 친구를 통해 서울지방법원에서 근...,"[김홍, 수, 게이트, 는, 90, 년대, 초, 초등, 학교, 동창, 이, ᆫ, 법..."
28599,89713,https://ko.wikipedia.org/wiki/%EC%9D%B8%EB%8F%...,인도아리아어군의 언어 목록,다음은 인도아리아어군의 언어 목록이다.\n\n\n 고전어\n 베다 산스크리트어\n ...,다음은 인도아리아어군의 언어 목록이다 고전어 베다 산스크리트어 산스크리트어 미탄니어...,"[다음, 은, 인도아리아어군, 의, 언어, 목록, 이, 다, 고, 전어, 베다, 산..."


### Lemmatization

In [8]:
def lemmatize_korean(tokens):
    lemmas = []
    remove_tags = {"Josa", "Eomi", "Suffix", "Punctuation"}

    for token in tokens:
        result = kiwi.analyze(token, top_n=1)
        if not result:
            continue

        for form, tag, *_ in result[0][0]:
            if tag not in remove_tags:
                lemmas.append(form)
                break
    return lemmas

df_sample_ko["tokens_lemmatized"] = df_sample_ko["tokens"].apply(lemmatize_korean)


In [9]:
df_sample_ko.head()

,id,url,title,text,clean_text,tokens,tokens_lemmatized
623706,3582873,https://ko.wikipedia.org/wiki/%EC%B9%B4%ED%83%...,카타쿠라 카네타로,카타쿠라 카네타로 ()는 현재의 카타쿠라 공업의 대대로 이어진 이름이다\n\n 카타...,카타쿠라 카네타로 는 현재의 카타쿠라 공업의 대대로 이어진 이름이다 카타쿠라 카네타...,"[카타쿠라 카네타로, 는, 현재, 의, 카타쿠라, 공업, 의, 대대, 로, 이어지,...","[카타쿠라 카네타로, 늘, 현재, 의, 카타쿠, 공업, 의, 대대, 로, 이어, ᆫ..."
23304,70654,https://ko.wikipedia.org/wiki/%ED%83%84%EB%91%90,탄두,"탄두(彈頭, )란 미사일, 로켓, 어뢰 등의 추진에 의해 전달되는 폭발성 물질 또는...",탄두 란 미사일 로켓 어뢰 등의 추진에 의해 전달되는 폭발성 물질 또는 독성 물질이...,"[탄두, 이, 란, 미사일, 로켓, 어뢰, 등, 의, 추진, 에, 의하, 어, 전달...","[탄두, 이, 란, 미사일, 로켓, 어뢰, 등, 의, 추진, 에, 의하, 어, 전달..."
79240,268688,https://ko.wikipedia.org/wiki/%EB%8B%88%EC%8A%...,니스 (프랑스),"니스(, 니사르드 , , , , )는 프랑스 남부의 항만 도시로 프랑스의 지중해 연...",니스 니사르드 는 프랑스 남부의 항만 도시로 프랑스의 지중해 연안에 위치해 있다 마...,"[니스, 니사르드, 는, 프랑스, 남부, 의, 항만, 도시, 로, 프랑스, 의, 지...","[니스, 니사르드, 늘, 프랑스, 남부, 의, 항만, 도시, 로, 프랑스, 의, 지..."
140214,529155,https://ko.wikipedia.org/wiki/%EA%B9%80%ED%99%...,김홍수 게이트,김홍수 게이트는 90년대 초 초등학교 동창인 법조인 친구를 통해 서울지방법원에서 근...,김홍수 게이트는 90년대 초 초등학교 동창인 법조인 친구를 통해 서울지방법원에서 근...,"[김홍, 수, 게이트, 는, 90, 년대, 초, 초등, 학교, 동창, 이, ᆫ, 법...","[김홍, 수, 게이트, 늘, 90, 년대, 초, 초등, 학교, 동창, 이, ᆫ, 법..."
28599,89713,https://ko.wikipedia.org/wiki/%EC%9D%B8%EB%8F%...,인도아리아어군의 언어 목록,다음은 인도아리아어군의 언어 목록이다.\n\n\n 고전어\n 베다 산스크리트어\n ...,다음은 인도아리아어군의 언어 목록이다 고전어 베다 산스크리트어 산스크리트어 미탄니어...,"[다음, 은, 인도아리아어군, 의, 언어, 목록, 이, 다, 고, 전어, 베다, 산...","[다음, 은, 인도아리아어군, 의, 언어, 목록, 이, 다, 고, 전어, 베, 산스..."


### Normalization

In [10]:
def load_stopwords(language: str):
    filename = f"stopwords/stopwords-{language}.txt"
    try:
        with open(filename, "r", encoding="utf-8") as f:
            stopwords = set(line.strip() for line in f if line.strip())
        return stopwords
    except FileNotFoundError:
        print(f"'{language}' - language not supported")
        return set()
    
stopwords_korean = load_stopwords("ko")

In [11]:
# Token Normalization Function (Stopword Removal)
stopwords_dict = {
    "ko": set(stopwords_korean),
    # "pt": set(stopwords_portuguese),
    # "it": set(stopwords_italian),
    # "en": set(stopwords_english),
}

def normalize_tokens(tokens, lang):
    if lang in stopwords_dict:
        stops = stopwords_dict[lang]
    else:
        from nltk.corpus import stopwords
        try:
            stops = set(stopwords.words(lang))  # NLTK stopwords for other languages
        except:
            stops = set()  # Fallback if stopwords are unavailable
    # Remove stopwords and single-character tokens
    normalized_tokens = [w for w in tokens if w not in stops and len(w) > 1]

    return normalized_tokens

# Apply Stopword Removal to the Pre-Tokenized Dataset
df_sample_ko["tokens_norm"] = df_sample_ko["tokens_lemmatized"].apply(lambda x: normalize_tokens(x, lang='ko'))

df_sample_ko.head()

,id,url,title,text,clean_text,tokens,tokens_lemmatized,tokens_norm
623706,3582873,https://ko.wikipedia.org/wiki/%EC%B9%B4%ED%83%...,카타쿠라 카네타로,카타쿠라 카네타로 ()는 현재의 카타쿠라 공업의 대대로 이어진 이름이다\n\n 카타...,카타쿠라 카네타로 는 현재의 카타쿠라 공업의 대대로 이어진 이름이다 카타쿠라 카네타...,"[카타쿠라 카네타로, 는, 현재, 의, 카타쿠라, 공업, 의, 대대, 로, 이어지,...","[카타쿠라 카네타로, 늘, 현재, 의, 카타쿠, 공업, 의, 대대, 로, 이어, ᆫ...","[카타쿠라 카네타로, 현재, 카타쿠, 공업, 대대, 이어, 이름, 카타쿠라 카네타로..."
23304,70654,https://ko.wikipedia.org/wiki/%ED%83%84%EB%91%90,탄두,"탄두(彈頭, )란 미사일, 로켓, 어뢰 등의 추진에 의해 전달되는 폭발성 물질 또는...",탄두 란 미사일 로켓 어뢰 등의 추진에 의해 전달되는 폭발성 물질 또는 독성 물질이...,"[탄두, 이, 란, 미사일, 로켓, 어뢰, 등, 의, 추진, 에, 의하, 어, 전달...","[탄두, 이, 란, 미사일, 로켓, 어뢰, 등, 의, 추진, 에, 의하, 어, 전달...","[탄두, 미사일, 로켓, 어뢰, 추진, 의하, 전달, 폭발, 물질, 또는, 물질, ..."
79240,268688,https://ko.wikipedia.org/wiki/%EB%8B%88%EC%8A%...,니스 (프랑스),"니스(, 니사르드 , , , , )는 프랑스 남부의 항만 도시로 프랑스의 지중해 연...",니스 니사르드 는 프랑스 남부의 항만 도시로 프랑스의 지중해 연안에 위치해 있다 마...,"[니스, 니사르드, 는, 프랑스, 남부, 의, 항만, 도시, 로, 프랑스, 의, 지...","[니스, 니사르드, 늘, 프랑스, 남부, 의, 항만, 도시, 로, 프랑스, 의, 지...","[니스, 니사르드, 프랑스, 남부, 항만, 도시, 프랑스, 지중해, 연안, 위치, ..."
140214,529155,https://ko.wikipedia.org/wiki/%EA%B9%80%ED%99%...,김홍수 게이트,김홍수 게이트는 90년대 초 초등학교 동창인 법조인 친구를 통해 서울지방법원에서 근...,김홍수 게이트는 90년대 초 초등학교 동창인 법조인 친구를 통해 서울지방법원에서 근...,"[김홍, 수, 게이트, 는, 90, 년대, 초, 초등, 학교, 동창, 이, ᆫ, 법...","[김홍, 수, 게이트, 늘, 90, 년대, 초, 초등, 학교, 동창, 이, ᆫ, 법...","[김홍, 게이트, 90, 년대, 초등, 학교, 동창, 법조인, 친구, 통하, 서울,..."
28599,89713,https://ko.wikipedia.org/wiki/%EC%9D%B8%EB%8F%...,인도아리아어군의 언어 목록,다음은 인도아리아어군의 언어 목록이다.\n\n\n 고전어\n 베다 산스크리트어\n ...,다음은 인도아리아어군의 언어 목록이다 고전어 베다 산스크리트어 산스크리트어 미탄니어...,"[다음, 은, 인도아리아어군, 의, 언어, 목록, 이, 다, 고, 전어, 베다, 산...","[다음, 은, 인도아리아어군, 의, 언어, 목록, 이, 다, 고, 전어, 베, 산스...","[인도아리아어군, 언어, 목록, 전어, 산스크리트어, 산스크리트어, 미탄니, 프라크..."


### Saving in CoNLL format

In [12]:
import stanza
from stanza.utils.conll import CoNLL
import os

def process_and_save_conllu(df, lang):

    stanza.download(lang)
    processors = "tokenize,pos,lemma"
    nlp_stanza = stanza.Pipeline(lang=lang, processors=processors, use_gpu=True )
    text = "\n".join(" ".join(tokens) for tokens in df["tokens_norm"])
    doc = nlp_stanza(text)

    os.makedirs("data", exist_ok=True)
    output_path = f"data/output_{lang}.conllu"
    CoNLL.write_doc2conll(doc, output_path)

process_and_save_conllu(df_sample_ko, lang="ko")


2025-10-29 19:02:40 INFO: Downloaded file to /Users/park/stanza_resources/resources.json
2025-10-29 19:02:40 INFO: Downloading default packages for language: ko (Korean) ...
2025-10-29 19:02:41 INFO: File exists: /Users/park/stanza_resources/ko/default.zip
2025-10-29 19:02:42 INFO: Finished downloading models and saved to /Users/park/stanza_resources
2025-10-29 19:02:42 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-10-29 19:02:42 INFO: Downloaded file to /Users/park/stanza_resources/resources.json
2025-10-29 19:02:42 INFO: Loading these models for language: ko (Korean):
| Processor | Package        |
------------------------------
| tokenize  | kaist          |
| pos       | kaist_nocharlm |
| lemma     | kaist_nocharlm |

2025-10-29 19:02:42 WARNING: GPU requested, but is not available!
2025-10-29 19:02:42 INFO: Using device: cpu
2025